Automação Web e Busca de Informações com Python
Desafio:
Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:

Dólar
Euro
Ouro
Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Para isso, vamos criar uma automação web:

Usaremos o selenium
Importante: baixar o webdriver

In [1]:
# Selenium: pacote de código python criado para automatizar tarefas na internet é necessário baixar o webdriver
# para firefox o driver é o geckodriver
# para o chrome é o driver chromedriver
# !pip install selenium

# Passo a Passo
# Passo 1: Entrar no navegador - Abrir o navegador
from selenium import webdriver # dentro do selenium existem várias funcionalidades e neste caso será utilizado apenas a webdriver por isso importa somente ela

navegador = webdriver.Chrome() # abre o navegador em segundo plano
navegador.get("https://www.google.com/") # precisa informar o endereço completo

In [2]:
# Passo 2: Importar a base de dados
import pandas as pd

tabela = pd.read_excel("commodities.xlsx")
display(tabela)


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN
5,Açúcar,136.23,NaN,NaN
6,Café,1092.87,NaN,NaN
7,Ouro,321.77,NaN,NaN
8,Tilápia,9.05,NaN,NaN


In [3]:
# .send_keys("ZZZZZZ") -> Se quiser escrever no campo
# .click() -> Se quiser clicar nele
# .get_attribute() -> pegar a informação contida nele 

import unicodedata
# Passo 3: Para cada produto
# Passo 4: Pegar o preço atual do produto
# Passo 5: Atualizar o preço na base de dados
for linha in tabela.index: 
    
    produto = tabela.loc[linha, "Produto"] # variável para percorrer toda a coluna de produto, essa variável será utilizada para montar o link

    # entrar no site melhor cambio para cada produto
    link = f"https://www.melhorcambio.com/{produto}-hoje" # precisa informar o endereço completo a utilização da varíavel produto para montar o link dinâmico
    link = link.replace("á","a").replace("é","e").replace("ã","a").replace("ó","o").replace("ú","u").replace("ç","c")
    #link = unicodedata.normalize("NFKD", link).encode("ascii","ignore")
    print(link)
    navegador.get(link) 

    # pegar a cotação atual do milho
    cotacao = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value') # deve ser informado no caminho do navegador onde essa informação pode ser obtida
    cotacao = cotacao.replace(".","")
    cotacao = cotacao.replace(",",".")
    cotacao = float(cotacao)
    print(cotacao)

    # na coluna preço atual do milho preencher com o valor informado no site.
    tabela.loc[linha,"Preço Atual"] = cotacao # tabela.loc[linha,coluna] ou seja uma célula


https://www.melhorcambio.com/Milho-hoje
53.72
https://www.melhorcambio.com/Soja-hoje
131.21
https://www.melhorcambio.com/Boi-hoje
248.3
https://www.melhorcambio.com/Petroleo-hoje
378.78
https://www.melhorcambio.com/Algodao-hoje
407.68
https://www.melhorcambio.com/Acucar-hoje
148.91
https://www.melhorcambio.com/Cafe-hoje
993.37
https://www.melhorcambio.com/Ouro-hoje
313.1
https://www.melhorcambio.com/Tilapia-hoje
9.76


In [4]:
# Passo 6: Decidir quais produtos deverão ser comprados
display(tabela)

tabela["Comprar"] = tabela["Preço Atual"] < tabela["Preço Ideal"] #analisa/aplica toda a coluna
display(tabela)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,53.72,NaN
1,Soja,163.59,131.21,NaN
2,Boi,282.20,248.30,NaN
3,Petróleo,424.37,378.78,NaN
4,Algodão,497.76,407.68,NaN
5,Açúcar,136.23,148.91,NaN
6,Café,1092.87,993.37,NaN
7,Ouro,321.77,313.10,NaN
8,Tilápia,9.05,9.76,NaN


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,53.72,True
1,Soja,163.59,131.21,True
2,Boi,282.20,248.30,True
3,Petróleo,424.37,378.78,True
4,Algodão,497.76,407.68,True
5,Açúcar,136.23,148.91,False
6,Café,1092.87,993.37,True
7,Ouro,321.77,313.10,True
8,Tilápia,9.05,9.76,False


In [5]:
# fechar o navegador
navegador.quit()

In [6]:
# Passo 7: Exportar a base de dados atualizada

tabela.to_excel("commodites_atualizado.xlsx", index=False) 